# Q1
Consider the discrete-time asset allocation example in section 8.4 of Rao and Jelvis. Suppose the single-time-step return of the risky asset as $Y_t=a$, prob=$p$ and $b$, prob=$(1-p)$. Suppose that T=10, use the TD method to find the Q function, and hence the optimal strategy.

# Answer

In this question, similar as the solution in section 8.4, we formulate the asset allocation problem as an MDP.

In the MDP, Reward is the final utility of the total wealth as time $T$ and is zero before $T$, and $gamma$ is set to be zero. The utility funcion is CARA and the reward is simplified to $\frac{-e^{-\alpha W_T}}{\alpha}$.

To reduce the complexity of TD learning, the problem is not modeled as an MDP with continuous state and action. I set the state as time ($s=0,1,...,T-1$) and discretize the actions (e.g., $a_t\in [0, 0.25, 0.5, 0.75, 1.0]$). Note that in a certain asset allocation problem, the wealth $W_t$ is highly positively correlated with time $t$, especially while the policy gradually converges to the optimal in TD method. Therefore such simplification is justified.

The implementation of the asset allocation is as below. The initial wealth $w_0=100$ and coefficency of CARA $\alpha=1e-4$ are fixed since these two parameters are not the focus of the question.

## Environment

In [ ]:
# environment
import numpy as np
class Asset:
  

    def __init__(self, a, b, p, r, action_space):
        
        """
        Initialize the asset investment environment.

        Parameters:
        a (float): High return of the risky asset.
        b (float): Low return of the risky asset.
        p (float): Probability of achieving high return a.
        r (float): Fixed return of the risk-free asset.
        action_space (list): Possible investment proportions (values between 0 and 1).
        """

        self.w0 = 100
        self.w = self.w0
        self.a = a
        self.b = b
        self.p = p
        self.r = r
        self.T = 10
        self.t = 0
        self.action_space = action_space

        self.alpha = 1e-4
        self.num_actions = len(self.action_space)

    def reset(self):

        """
        Reset the environment to the initial state.

        Returns:
        - t (int): Current time step (0).
        """
        
        self.t = 0
        self.w = self.w0
        return self.t

    def utility(self, w):

        """
        Compute the CARA (Constant Absolute Risk Aversion) utility function value.

        Parameters:
        w (float): Wealth value.

        Returns:
        - utility (float): Computed CARA utility value.
        """
        
        return (1 - np.exp(-self.alpha * w)) / self.alpha

    def step(self, action):

        """
        Perform one step of investment.

        Parameters:
        action (int): Index of the chosen investment proportion (from action_space).

        Returns:
        - next_t (int): Next time step.
        - new_w (float): Updated wealth.
        - reward (float): Immediate reward (utility value at final step).
        - done (bool): Whether the final time step T has been reached.
        """

        action_val = np.round(self.action_space[action], 2)
        risk = self.w * action_val
        riskless = self.w * (1 - action_val)
        gain = riskless * self.r
        if np.random.rand() < self.p:
            gain += self.a * risk
        else:
            gain += self.b * risk
        self.t += 1
        self.w += gain
        done = self.t >= self.T
        if done:
            reward = self.utility(self.w)
        else:
            reward = 0

        done = self.t >= self.T
        return self.t, self.w, reward, done

## TD method

The following is the implementation of the TD method.

In [ ]:
def argmax_w_rand(a):

    """
    Handle tie-breaking when selecting the action with the highest Q-value.

    Parameters:
    a (numpy.ndarray): 1D array of Q-values.

    Returns:
    - index (int): The index of the action with the highest Q-value (randomly chosen if there is a tie).
    """

    maxval = np.max(a)
    ind = np.flatnonzero(np.abs(a - maxval) < 1e-6)
    return np.random.choice(ind)

def td_learning(env, lr, num_episodes, eps=0.1, min_eps=0.01):


    """
    Train the Q-values using the Temporal Difference (TD) method.

    Parameters:
    env (Asset): The investment environment.
    lr (float): Learning rate (e.g., 0.05).
    num_episodes (int): Number of training episodes.
    eps (float): Initial exploration rate (default 0.1).
    min_eps (float): Minimum exploration rate (default 0.01).

    Returns:
    - Q (numpy.ndarray): Trained Q-value table, shape (T, num_actions).
    """

    Q = np.zeros((env.T, env.num_actions))
    init_eps = eps
    eps_decay = (init_eps - min_eps) / num_episodes
    for _ in range(num_episodes):
        state = env.reset()
        eps -= eps_decay
        done = False
        if np.random.uniform(0, 1) < eps:
            action = np.random.randint(0, env.num_actions)  # explore
        else:
            action = argmax_w_rand(Q[state, :])  # exploit
        while not done:
            next_state, _, reward, done = env.step(action)
            if np.random.uniform(0, 1) < eps:
                next_action = np.random.randint(0, env.num_actions)  # explore
            else:
                next_action = argmax_w_rand(Q[state, :])  # exploit

            # print(state, action, reward, next_state, next_action)
            Q[state, action] = Q[state, action] + lr * (reward + (0 if done else Q[next_state, next_action]) - Q[state, action])
            state = next_state
            action = next_action
    return Q


## Test cases and observation

Next I solve the problem using TD method and show the optimal policy, for several different prblems.

### 1.
In the first test case, the risky asset always have higher profit than riskless asset, i.e., $a>r,b>r$.

In [3]:
np.random.seed(0)
asset = Asset(a=0.8, b=0.5, p=0.4, r=0.1, action_space=np.round(np.linspace(0, 1, 5), 1))
Q = td_learning(asset, lr=0.05, num_episodes=10000)

print([np.argmax(Q[s, :]) for s in range(10)])
print(Q)

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[[4634.52744618 4368.01937422 4649.93931317 4421.56264534 6746.02112798]
 [4619.83807321 4640.75955431 4534.40077729 3901.23341292 6717.62528097]
 [4825.62321671 4415.06121472 4619.42848397 5322.86409592 6717.47472387]
 [5114.69682341 5175.22737552 5041.22949117 5262.54630758 6739.62729568]
 [4991.79737345 4783.82804074 5142.73152417 5425.01012636 6762.68310446]
 [4668.89861716 5215.9035607  5458.2256218  5550.13365099 6802.70649498]
 [5339.54274355 5386.32966612 5421.58684627 4748.77295084 6855.94464033]
 [5244.65677146 5742.34056492 6186.34524581 5361.57456344 6888.85649504]
 [5168.60898695 5951.32180012 6364.22377011 5720.87366885 6843.11128165]
 [4237.02911427 5293.59524993 6355.72182622 5216.26481831 6757.70116829]]


The optimal policy is to always invest the risky asset (action 4).

### 2.
In the second test case, the risky asset always have lower profit than riskless asset, i.e., $a<r,b<r$.

In [4]:
np.random.seed(0)
asset = Asset(a=0.1, b=0.1, p=1.0, r=0.8, action_space=np.round(np.linspace(0, 1, 5), 1))
Q = td_learning(asset, lr=0.05, num_episodes=10000)

print([np.argmax(Q[s, :]) for s in range(10)])
print(Q)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[9447.15218726 4850.23302107 5362.85563022 5377.19027604 5127.99985943]
 [9403.70377373 5355.72412507 5225.70847917 4490.57678701 5297.55754413]
 [9415.1643229  5044.6056985  5236.07947474 6270.09574767 6210.02271454]
 [9479.21070857 6274.17842612 6276.8517519  6273.53724009 5260.78627853]
 [9539.8298406  6092.30229417 6264.17156036 6617.64119565 5869.74981775]
 [9592.71270888 5664.8199922  5433.14059908 7165.27176809 7219.55832445]
 [9617.4736983  5629.79153337 6970.03011963 7071.41549468 6211.23467122]
 [9631.42177017 6636.36055422 7774.78622961 7446.45904648 4578.11951788]
 [9661.1433327  6662.50682782 8277.53927636 7512.22118938 5994.50286411]
 [9706.37769302 6993.78580221 9234.00961151 8158.19822998 5369.33990109]]


The optimal policy is to always invest the riskless asset (action 0).

## 3.
In the third test case, $r=0.4, a=0.6, b=0.2, p=0.5$.

In [5]:
np.random.seed(0)
asset = Asset(a=0.6, b=0.2, p=0.5, r=0.4, action_space=np.round(np.linspace(0, 1, 5), 1))
Q = td_learning(asset, lr=0.05, num_episodes=10000)

print([np.argmax(Q[s, :]) for s in range(10)])
print(Q)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[2506.46653546 2402.80877392 2417.0656036  2417.90367283 2400.42449458]
 [2508.05713943 2410.99449907 2401.16184529 2387.71638856 2408.86532236]
 [2510.54345136 2442.7628125  2462.72895674 2457.98429952 2463.58298051]
 [2513.61300226 2475.15657725 2477.76422275 2467.4739422  2451.27354015]
 [2516.47345206 2476.86493914 2483.51420624 2470.4056505  2465.12936905]
 [2519.22638952 2476.08597521 2492.94021628 2483.48850049 2471.68649132]
 [2522.52658685 2451.290506   2508.37168601 2490.37676716 2482.18979839]
 [2526.81408625 2486.42534187 2518.60885697 2505.7092074  2501.42260052]
 [2530.91017987 2501.83903002 2529.90272797 2525.85054407 2524.11009052]
 [2508.68350439 2589.09124388 2547.12337189 2527.66308548 2551.52970628]]


The policy tends to invest riskless asset at most times, though the expectation yields are the same, the risky asset has higher variance, which leads to lower utility.

### 4.
In the fourth test case, $r=0.4,a=0.8,b=0.2,p=0.5$.

In [6]:
np.random.seed(0)
asset = Asset(a=0.8, b=0.2, p=0.5, r=0.4, action_space=np.round(np.linspace(0, 1, 5), 1))
Q = td_learning(asset, lr=0.05, num_episodes=50000)

print([np.argmax(Q[s, :]) for s in range(10)])
print(Q)

[4, 4, 4, 4, 4, 4, 4, 4, 4, 3]
[[3473.91335218 3720.72827511 3629.10207621 3636.54169203 3880.5431668 ]
 [3536.72751874 3573.18442985 3527.37300378 3626.70522784 3857.43683744]
 [3577.67525926 3609.72854113 3672.53539752 3710.36520054 3840.15190556]
 [3651.71248488 3645.91034929 3648.08270692 3620.90691601 3833.44614107]
 [3591.19608131 3645.09236454 3602.2448913  3698.25160371 3833.78927459]
 [3722.31531379 3739.39642577 3648.50074148 3685.39929136 3854.14346839]
 [3551.20218854 3776.1690241  3687.16781998 3751.00595378 3915.95804921]
 [3785.80921587 3803.81622368 3661.99514299 3606.36767882 4040.29513131]
 [3856.06915095 3859.27981127 3694.88327763 3862.47778424 4222.38305671]
 [3905.12853326 3592.58712804 3762.6631491  4273.0841716  4224.57142404]]


The learnt policy tends to invest risky asset at most times. We can notice that only $a$ is changed compared to the third case. Higher expectation of profit makes the risky asset better than the riskless asset in this case.

### 5.
The above are rather extreme cases. In the fifth test case, $r=0.15,a=0.3,b=0.1,p=0.4$, which means the risky asset has slightly higher expectation profit (0.18) than the riskless asset (0.15).

In [7]:
np.random.seed(0)
asset = Asset(a=0.3, b=0.1, p=0.4, r=0.15, action_space=np.round(np.linspace(0, 1, 5), 1))
Q = td_learning(asset, lr=0.05, num_episodes=10000)

print([np.argmax(Q[s, :]) for s in range(10)])
print(Q)

[4, 4, 4, 4, 4, 4, 4, 4, 3, 2]
[[451.58288349 446.60122618 452.29525622 455.45353621 498.80780109]
 [448.01741979 451.40739798 448.52139242 446.32553446 496.616281  ]
 [458.66775082 447.74332826 456.02081115 450.16036951 495.29225515]
 [443.95343834 454.62094417 465.50059557 454.27406923 494.98127465]
 [459.59409588 452.59003143 467.09487702 471.54842905 495.91438123]
 [456.23579706 461.05175272 457.02847203 475.52564647 498.95343692]
 [475.12852371 451.89035336 474.25021032 488.07125452 503.48631048]
 [451.76916969 464.35544066 479.09984936 493.1971476  505.36687861]
 [466.6199084  454.27275229 484.60530898 516.18452099 497.29622329]
 [479.39703224 483.96975884 505.07327663 490.53095192 494.14256363]]


The optimal policy turns out to be investing as much as possible in earlier steps and turns conservative later (due to the CARA utility).